# Introduction aux transformations avec XSLT

## Présentation

**XSL(T) :** *eXtensible Stylesheet Language Transformations*

D’un point de vue formel, le langage XSLT est une grammaire écrite en XML qui utilise le langage XPath pour naviguer parmi les nœuds d’un document structuré. Il est à considérer comme un équivalent du CSS pour le langage HTML, en ce sens qu’il va transformer le document source pour en fournir une autre version. Il s’agit en plus d’une [recommandation du W3C](https://www.w3.org/TR/xslt-30/)

**23 novembre 1999 :** version 1.0

**23 janvier 2007 :** version 2.0

**8 juin 2017 :** version 3.0

### Principe d’une transformation

XSLT produit en sortie un document modélisable sous forme d’arbre (XML, HTML, PDF, RTF, SVG…) :

![Schéma de transformation d’un document structuré vers un autre document structuré](./pics/1-fig1.png)

### Modèle d’une feuille de transformation

```xml
<?xml version="1.0" encoding="UTF-8"?>

<!-- root element -->
<xsl:stylesheet version="3.0" xmlns:xsl="http://www.w3.org/1999/XSL/Transform">

    <!-- indication on the document to be produced -->
    <xsl:output method="xml" version="1.0" encoding="utf-8" indent="yes"/>

    <!-- main transformation rule -->
    <xsl:template match="/">

        <!-- backbone of the new document -->
        <nvXML>
            <!-- call for application -->
            <xsl:apply-templates select="{XPath}"/>
        </nvXML>

    </xsl:template>

    <!-- candidates -->
    <xsl:template match="{XPath}"></xsl:template>
    <xsl:template match="{XPath}"></xsl:template>

</xsl:stylesheet>
```

## Produire un nouveau document XML

Transformer un document XML en un autre document XML. L’intérêt peut au premier abord sembler obscur. Pourtant, les occasions d’harmoniser un ensemble de documents variés pour les intégrer dans un système cohérent ne manquent pas :
- catalogues de fournisseurs ;
- procédures de transferts bancaires ;
- partage de calendriers ;
- export en divers formats
- …

Le fichier *bbt-simple.xml* reprend un fragment XML simplissime :

```xml
<?xml version="1.0" encoding="UTF-8"?>
<show title="The Big Bang Theory">
    <author>Chuck Lorre</author>
    <author>Bill Prady</author>
    <year>2007</year>
</show>
```

L’objectif ici est de le transformer dans un autre format :

```xml
<?xml version="1.0" encoding="UTF-8"?>
<bbt>
    <createur>Chuck Lorre</createur>
    <createur>Bill Prady</createur>
    <annee>2007</annee>
</bbt>
```

### 1e étape : écrire la feuille de transformation minimale

```xml
<?xml version="1.0" encoding="UTF-8"?>

<!-- root element -->
<xsl:stylesheet version="3.0" xmlns:xsl="http://www.w3.org/1999/XSL/Transform">

    <!-- an XML document will be produced -->
    <xsl:output method="xml" version="1.0" encoding="utf-8" indent="yes"/>

    <!-- backbone -->
    <xsl:template match="/">
        <!-- main transformation rule -->
    </xsl:template>

</xsl:stylesheet>
```

### 2e étape : décrire la règle principale de transformation

```xml
<!-- backbone -->
<xsl:template match="/">
    <!-- root element of the output -->
    <bbt>
        <!-- call for application for nodes 'author' -->
        <xsl:apply-templates select="//author"/>
        <!-- call for application for nodes 'year' (only one) -->
        <xsl:apply-templates select="//year"/>
    </bbt>
</xsl:template>
```

### 3e étape : description des templates candidats

```xml
<!-- candidate 'author' -->
<xsl:template match="author">

    <!-- an element 'creator' will be created -->
    <createur>
        <!-- copy of the textual value inside 'author' -->
        <xsl:value-of select="."/>
    </createur>

</xsl:template>

<!-- candidate 'year' -->
<xsl:template match="year">

    <!-- a new element 'annee' -->
    <annee>
        <!-- copy of the textual value -->
        <xsl:value-of select="."/>
    </annee>

</xsl:template>
```

### Lancer la transformation avec le module *lxml*

Le module *lxml* est un autre module Python qui permet de manipuler des documents XML tout en offrant un support pour XSLT. Il est à noter qu’il est en plus compatible avec les commandes de base du module  *elementTree* :

In [ ]:
from lxml import etree

tree = etree.parse('./data/bbt-simple.xml')
root = tree.getroot()

for child in root:
    print(child.tag)

Comme un document XSLT est écrit en XML, le même méthode `parse()` convient pour l’analyser :

In [ ]:
xslt = etree.parse('./data/bbt-simple.xsl')

L’étape d’après consiste à instancier un processeur XSLT avec la méthode `XSLT()` :

In [ ]:
processor = etree.XSLT(xslt)

Pour finalement exécuter la transformation avec le processeur :

In [ ]:
new_tree = processor(tree)

Vérifiez le résultat :

In [ ]:
print(etree.tostring(new_tree))

Et pour enregistrer le nouvel arbre XML dans un fichier, il suffit d’utiliser la méthode `write()` :

In [ ]:
new_tree.write('./data/new-bbt-simple.xml')

À noter le paramètre nommé `pretty_print` qui permet d’établir automatiquement l’indentation des balises :

In [ ]:
new_tree.write('./data/new-bbt-simple.xml', pretty_print=True)

## Produire un document HTML

Un autre des intérêts de transformer un document XML en un autre document XML est lorsque l’on souhaite se reposer sur un document XML pour créer une page HTML.

À partir du même document XML, l’objectif est maintenant de produire un fichier *big-bang-theory.html* :

```html
<!DOCTYPE html>
<html lang="fr" xmlns="http://www.w3.org/1999/xhtml">
<head>
    <meta charset="utf-8" />
    <title>Présentation | The Big Bang Theory</title>
</head>
<body>
    <h1>The Big Bang Theory</h1>
    <p><b>Créateurs :</b> Chuck Lorre, Bill Prady<br />
    <b>Année de création :</b> 2007</p>
</body>
</html>
```

### 1e étape : écrire la feuille de transformation minimale

```xml
<?xml version="1.0" encoding="UTF-8"?>
<xsl:stylesheet version="3.0" xmlns:xsl="http://www.w3.org/1999/XSL/Transform">

    <!-- an HTML document will be produced -->
    <xsl:output method="html" encoding="utf-8" indent="yes"/>

    <!-- backbone -->
    <xsl:template match="/">

        <!-- special root to print HTML doctype -->
        <xsl:text disable-output-escaping="yes">&amp;lt;!DOCTYPE html&amp;gt;</xsl:text>
        <xsl:text>&amp;#xA;</xsl:text>

    </xsl:template>

</xsl:stylesheet>
```

### 2e étape : décrire la règle principale de transformation

```xml
<!-- backbone -->
<xsl:template match="/">

    <xsl:text disable-output-escaping="yes">&amp;lt;!DOCTYPE html&amp;gt;</xsl:text>
    <xsl:text>&amp;#xA;</xsl:text>

    <html lang="fr" xmlns="http://www.w3.org/1999/xhtml">
        <head>
            <title>Présentation | <xsl:apply-templates select="show/@title"/></title>
            <meta charset="utf-8"/>
        </head>
        <body>
            <!-- call for application for attribute 'title', child of element 'show' -->
            <h1><xsl:apply-templates select="show/@title"/></h1>
            <p>
                <b>Créateurs :</b>
                <xsl:text> </xsl:text>
                <!-- call for nodes 'author' -->
                <xsl:apply-templates select="//author"/>
                <br/>
                <b>Année de création :</b>
                <xsl:text> </xsl:text>
                <!-- call for nodes 'year' (only one) -->
                <xsl:apply-templates select="//year"/>
            </p>
        </body>
    </html>

</xsl:template>
```

### 3e étape : description des templates candidats

```xml
<!-- a template than can apply to all calls -->
<xsl:template match="*">
    <!-- copy the textual content of the current node -->
    <xsl:value-of select="."/>
</xsl:template>

<!-- cadidate 'author' -->
<xsl:template match="author">
    <!-- copy the textual content of the current node -->
    <xsl:value-of select="."/>
    <!-- if the current node is not the last one, add a comma -->
    <xsl:if test="position() != last()">, </xsl:if>
</xsl:template>
```

### 4e étape : lancer la transformation

In [ ]:
from lxml import etree

tree = etree.parse('./data/bbt-simple.xml')
xslt = etree.parse('./data/bbt-to-html.xsl')
processor = etree.XSLT(xslt)
html = processor(tree)
html.write('./data/big-bang-theory.html')

## Principes de la transformation avec XSLT

### Un ensemble de règles

Les transformations respectent un modèle de traitement par ensemble de règles :

1. Sélection des noeuds correspondant au *pattern* ;
2. actions à effectuer pour chaque élément de la sélection ;
3. exécution du traitement.

```xml
<!-- this template is ready to apply to a call -->
<!-- the attribute 'match' expresses a condition:
<!-- from the current position of the pointer, an element 'actor' must exist -->
<xsl:template match="actor">
    <p>
        <b>Prénom :</b>
        <!-- copy the textual content of node 'firstname' -->
        <xsl:value-of select="firstname"/>
        <b>Nom :</b>
        <!-- copy the textual content of node 'lastname' -->
        <xsl:value-of select="lastname"/>
    </p>
</xsl:template>
```

### Une résolution non-linéaire des règles

Plutôt que d’exécuter les règles dans l’ordre de leur apparition dans le code, XSLT repose sur la notion de hiérarchie des instructions. La fonction `apply-templates` est alors chargée de déclencher le traitement des enfants liés au modèle :

```xml
<!-- candidate for node 'actor' -->
<xsl:template match="//actor">
    <!-- its role? call for suitable candidates -->
    <p><xsl:apply-templates/></p>
</xsl:template>

<!-- candidate for node 'firstname' -->
<xsl:template match="firstname">
    <b>Prénom :</b><xsl:value-of select="."/>
</xsl:template>

<!-- candidate for node 'lastname' -->
<xsl:template match="lastname">
    <b>Nom :</b><xsl:value-of select="."/>
</xsl:template>

<!-- candidate for node 'birth' -->
<xsl:template match="birth">
    <b>Date de naissance :</b>
    <xsl:value-of select="format-date(.,'[D01] [MNn] [Y0001]')"/>
</xsl:template>
```

### Rôles d’un template

1. Interroger l’emplacement du pointeur dans la modélisation sous forme arborescente du document XML.
2. Tester la validité de l’expression XPath (attribut `match`).
    - Si elle est éligible :
        - déplacer le pointeur au niveau de la résolution de l’expression XPath ;
        - exécuter les actions définies ;
        - replacer le pointeur à l’étape précédente.
    - Sinon, il passe son tour.

## Les variables en XSLT

### Enregistrer un résultat

XSLT peut stocker le résultat d’une expression XPath dans une variable, tout comme du texte simple :

```xml
<!-- a variable named 'jim' stores the result of an XPath expression -->
<xsl:variable name="jim" select="//actor[firstname='Jim']"/>
```

```xml
<xsl:variable name="copyright">
<footer>
    <p>© The Big Bang Theory, 2018</p>
</footer>
</xsl:variable>
```

Toute expression XPath peut servir à isoler un fragment d‘arbre ou à enregistrer un traitement :

```xml
<!-- a count -->
<xsl:variable name="nb" select="count(//character)"/>
<!-- a boolean -->
<xsl:variable name="isLeonard" select="//firstname='Leonard'"/>
```

### Appeler une variable

Une fois instanciée, une variable est référencée par le préfixe `$` :

```xml
<!-- only the textual content is copied -->
<xsl:value-of select="$copyright"/>
```

```xml
<!-- elements and attributes are also copied -->
<xsl:copy-of select="$copyright"/>
```

## Exécuter une transformation en ligne de commande

L’utilitaire qui applique une feuille de transformations à un document XML s’appelle `xsltproc`. Il est prévu pour fonctionner avec les spécifications XPath 1.1, aussi toute fonction XPath 2 ou d’une version supérieure provoquera une erreur bloquante. Si seul l’attribut `version` de la balise `stylesheet` indique le recours à une version supérieure à 1.1, l’erreur soulevée n’empêchera pas l’exécution de la feuille de transformations :

In [ ]:
!xsltproc ./data/bbt-to-html-full.xsl ./data/bbt.xml

Pour écrire le résultat dans un document, il suffit de rediriger le flux de sortie :

In [ ]:
!xsltproc ./data/bbt-to-html-full.xsl ./files/bbt.xml > ./files/bbt.html